In [1]:
DATA_PATH = '../../data/CRTS2/'

In [2]:
import pandas as pd

Load Transient Catalog

In [3]:
filename = 'transient_catalog.pickle'
indir = DATA_PATH; filepath = indir + filename
df_cat = pd.read_pickle(filepath)

Load Transient Lightcurves

In [4]:
filename = 'transient_lightcurves.pickle'
indir = DATA_PATH; filepath = indir + filename
df_lcs = pd.read_pickle(filepath)

In [5]:
df_cat.TransientID.unique().shape

(5539,)

In [6]:
df_lcs.TransientID.unique().shape

(4984,)

Check that all Lightcurves TransientID's are in Catalog

In [7]:
all_exist = True
transID_cat_list = df_cat.TransientID.unique()
for lcs_id in df_lcs.TransientID.unique():
    all_exist = all_exist and (lcs_id in transID_cat_list)
print('All exist:', str(all_exist))

All exist: True


Check that all Catalog TransientID's are in Lightcurves 

In [8]:
all_exist = True
lcs_missing_transientID_list = []
transID_lcs_list = df_lcs.TransientID.unique()
for cat_id in df_cat.TransientID.unique():
    id_exists = (cat_id in transID_lcs_list)
    all_exist = all_exist and id_exists
    if not id_exists: lcs_missing_transientID_list.append(cat_id)
print('All exist:', str(all_exist))
print('Missing: {} Transients'.format(len(lcs_missing_transientID_list)))

All exist: False
Missing: 555 Transients


Merge dataframes with inner join

In [9]:
df_merge = df_cat.copy().merge(df_lcs.copy().groupby('TransientID',as_index=False).count(), how='inner')
df_merge.rename(columns={'Mag':'ObsCount'}, inplace=True)
df_merge = df_merge[['TransientID', 'Classification', 'ObsCount']]

In [10]:
print(df_merge.shape)

(4985, 3)


Top 10 count of transients by class

In [42]:
df = df_merge[['Classification','ObsCount']].groupby('Classification').count()
df = df.rename(columns={'ObsCount':'ObjCount'}).sort_values('ObjCount', ascending=False)
print(df.iloc[:10])

                ObjCount
Classification          
SN                  1539
CV                   943
HPM                  437
AGN                  429
SN?                  294
Blazar               238
Flare                215
Unknown              186
AGN?                 133
CV?                   68


Count number of objects with at least 5 observations

In [12]:
df_merge_filtered = df_merge[df_merge.ObsCount >= 5]
df_merge_filtered.shape[0]

4385

Top 10 count of transients by class (with at least 5 observations)

In [43]:
df = df_merge_filtered[['Classification','ObsCount']].groupby('Classification').count()
df = df.rename(columns={'ObsCount':'ObjCount'}).sort_values('ObjCount', ascending=False)
print(df.iloc[:10])

                ObjCount
Classification          
SN                  1295
CV                   862
AGN                  427
HPM                  413
SN?                  239
Blazar               236
Flare                207
AGN?                 132
Unknown              112
CV?                   55
